In [1]:
## This script is to plot GPMF hourly over OMAN and total precipitation 
## Created by C. Bayu Risanto, S.J. (1 March 2025)
import warnings 
warnings.filterwarnings('ignore')
import os
import numpy as np
from numpy import matlib
import scipy as sp
from random import *
import math as mt
from math import * #cos,sin,asin,sqrt,pi,radians,atan2
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.ticker as mticker
import matplotlib.dates as mdates
#from mpl_toolkits.basemap import Basemap # map functionality
from matplotlib.patches import Polygon
from matplotlib import cm
from matplotlib.patches import Patch
import matplotlib.colors as mcolors
from matplotlib.colors import BoundaryNorm
from matplotlib.colors import ListedColormap,LinearSegmentedColormap
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.io.shapereader import Reader
from cartopy.feature import ShapelyFeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import xarray as xr
import netCDF4 as nc
from netCDF4 import Dataset
import pandas as pd
from datetime import date, datetime, timedelta
import sys
import glob

## opener
def readGP(ncfile):
    dx = xr.open_dataset(ncfile)
    pcp = dx.GPM_3IMERGHH_07_precipitation[0,:,:]
    lat = dx.lat[:]
    lon = dx.lon[:]
    return pcp,lat,lon
def read_nc(ncfile):
    dx = xr.open_dataset(ncfile)
    HGT = dx.HGT_M[0,:,:]
    lon = dx.XLONG_M[0,:,:]
    lat = dx.XLAT_M[0,:,:]
    return HGT,lon,lat

In [2]:
## Get Time of interests
sdate = '202404130000'
edate = '202404162330'
datestrform = '%Y%m%d%H%M'
date_st = datetime.strptime(sdate,datestrform)-timedelta(hours=0.5)
date_en = datetime.strptime(edate,datestrform)
dateList = [date_st.strftime(datestrform)]
date_time = date_st; dateList = []
while date_time < date_en:
    date_time += timedelta(hours=0.5)
    dateList.append(date_time.strftime(datestrform))
dateList = np.asarray(dateList)


In [3]:
## Read data GPMF
link = '/home/bayu/DATA/LIRIC/OMAN/observed/GPMF_hr/'
pcp_al = []
for i in range(len(dateList)):
    ncfile = 'g4.subsetted.GPM_3IMERGHH_07_precipitation.'+dateList[i]+'00.49E_15N_64E_27N.nc'
    pcp,_,_ = readGP(link+ncfile)
    pcp_al.append(pcp)
al_pcp = np.asarray(pcp_al)

In [4]:
## get lat lon
_,lat,lon = readGP(link+ncfile)
xlon = np.matlib.repmat(lon,len(lat),1)    ; #print(xlon.shape)
xlat = np.matlib.repmat(lat,len(lon),1).T      ; #print(xlat.shape)

In [5]:
print(xlon.shape,xlat.shape,al_pcp.shape)

(121, 150) (121, 150) (192, 121, 150)


In [6]:
## read hgt
linkH = '/home/bayu/DATA/LIRIC/OMAN/geog/'
file = 'geo_em.d01.nc'
hgt,lonh,lath = read_nc(linkH+file)
## get rid of anything less than 0 m
hgt = hgt.where(hgt >= 1)

In [7]:
xticks = np.arange(50.,64.,1)
yticks = np.arange(16.,28.,1)
## create background
crs = ccrs.PlateCarree()
def plot_background(ax):
    ax.set_extent([50.,64.,16.,28.], ccrs.PlateCarree())
    ax.add_feature(cfeature.COASTLINE.with_scale('50m'),linewidth=1.4,zorder=3)
    ax.add_feature(cfeature.BORDERS,linestyle=':',linewidth=0.4,zorder=3)
    #ax.add_feature(cfeature.STATES,linestyle=':',linewidth=0.9,zorder=3)
    ax.set_xticks(xticks, crs=ccrs.PlateCarree())
    ax.set_yticks(yticks, crs=ccrs.PlateCarree())
    ax.set_xticklabels(xticks, rotation=0, fontsize=12)
    ax.set_yticklabels(yticks, rotation=0, fontsize=12)
    ax.xaxis.set_major_formatter(LongitudeFormatter(zero_direction_label=True))
    ax.yaxis.set_major_formatter(LatitudeFormatter())
    
    gl = ax.gridlines(ccrs.PlateCarree(),draw_labels=False,linewidth=2,color='gray',
                     alpha=0.5, linestyle='--')
    gl.xlabels_top = False
    gl.xlabels_bottom = False
    gl.ylabels_left = False
    gl.ylabels_right = False
    gl.xlines = False; gl.ylines = False
    gl.xlocator = mticker.FixedLocator(xticks)
    gl.ylocator = mticker.FixedLocator(yticks)
    
    return ax

In [8]:
# create Precip color
pcp_colors = ['#ffffff','#00eeee','#00b2ee','#1e90ff','#104e8b',
             '#7fff00','#00cd00','#008b00','#ffff00','#ffd700',
             '#cd8500','#ff7f00','#ee4000','#cd0000','#ec5959',
             '#ffaeb9','#8968cd','#912cee','#8b008b']

varval     = [0., 1, 2, 3, 5, 10, 15, 20, 25, 30, 35,
               40, 45, 50, 75, 100, 125, 150, 200, 300]

precip_cmap = mcolors.ListedColormap(pcp_colors,'precipitation')
adjnorm = mcolors.BoundaryNorm(varval,precip_cmap.N) 

var_hr = [0.,1, 2, 3, 4, 6, 8, 10, 12, 14, 16, 18, 20, 25, 
          30, 35, 40, 45, 50, 60]
precip_hr = mcolors.ListedColormap(pcp_colors,'precip_hr')
adjnorm_hr = mcolors.BoundaryNorm(var_hr,precip_hr.N)

In [9]:
## PLOT HERE
projection = ccrs.PlateCarree()
for i in range(len(dateList)):
    fig, ax1 = plt.subplots(1, 1, figsize=(10, 10),
                                   subplot_kw={'projection': projection})
## AX1
    plot_background(ax1)
## plot terrae height
    pcp1 = ax1.contourf(xlon,xlat,al_pcp[i,:,:],
                      var_hr,cmap=precip_hr,norm=adjnorm_hr,transform=projection)
    top1 = ax1.contour(lonh,lath,hgt,11,vmin=0,vmax=2500,colors='k',linewidths=0.6)

    ax1.set_title('GPMF Rain'+' at '+dateList[i]+' Z',
                         loc='center',pad=5,fontsize=14) 
#Add height colorbar
    cax = fig.colorbar(pcp1,ticks=var_hr,ax=ax1,orientation='horizontal',shrink=0.98,aspect=50,pad=0.04)
    cax.ax.tick_params(labelsize=16)

    lat_MCT = 23.5880
    lon_MCT = 58.3829 
    ax1.plot(lon_MCT,lat_MCT, marker='*',markerfacecolor='none', 
                 markeredgecolor='k', markersize=16, transform=projection)

## SAVE
    #dir_out = '/home/bayu/PLOTS/LIRIC/samples/'
    #plotfile  = 'OMAN_'+dateList[i]+'Z.png'
    #sf = fig.savefig(dir_out+plotfile, dpi=300, bbox_inches='tight')

## CLOSE
    plt.show()
    plt.close()

IndentationError: unexpected indent (3415988303.py, line 14)